# Analyzing data with Dask, SQL, and Coiled

In this notebook, we look at using [Dask-SQL](https://dask-sql.readthedocs.io/en/latest/), an exciting new open-source library which adds a SQL query layer on top of Dask. This allows you to query and transform Dask DataFrames using common SQL operations.

## Launch a cluster

Let's first start by creating a Coiled cluster which uses the `examples/dask-sql` software environment, which has `dask`, `pandas`, `s3fs`, and a few other libraries installed.

In [ ]:
import coiled

cluster = coiled.Cluster(
    n_workers=10,
    worker_memory="30GiB",
    software="examples/dask-sql",
)
cluster

and then connect Dask to our remote Coiled cluster

In [ ]:
from dask.distributed import Client

client = Client(cluster)
client.wait_for_workers(10)
client

## Getting started with Dask-SQL

Internally, Dask-SQL uses a well-established Java library, Apache Calcite, to parse SQL and perform some initial work on your query. To help Dask-SQL locate JVM shared libraries, we set the `JAVA_HOME` environment variable. 

In [ ]:
import os

os.environ["JAVA_HOME"] = os.environ["CONDA_DIR"]

The main interface for interacting with Dask-SQL is the `dask_sql.Context` object. It allows your to register Dask DataFrames as data sources and can convert SQL queries to Dask DataFrame operations.

In [ ]:
from dask_sql import Context

c = Context()

For this notebook, we'll use the NYC taxi dataset, which is publically accessible on AWS S3, as our data source

In [ ]:
import dask.dataframe as dd
from distributed import wait

df = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-*.csv",
    dtype={
        "payment_type": "UInt8",
        "VendorID": "UInt8",
        "passenger_count": "UInt8",
        "RatecodeID": "UInt8",
    },
    storage_options={"anon": True}
)

# Load datasest into the cluster's distributed memory.
# This isn't strictly necessary, but does allow us to
# avoid repeated running the same I/O operations. 
df = df.persist()
wait(df);

We can then use our `dask_sql.Context` to assign a table name to this DataFrame, and then use that table name within SQL queries

In [ ]:
# Registers our Dask DataFrame df as a table with the name "taxi"
c.register_dask_table(df, "taxi")

# Perform a SQL operation on the "taxi" table
result = c.sql("SELECT count(1) FROM taxi")
result

Note that this returned another Dask DataFrame and no computation has been run yet. This is similar to other Dask DataFrame operations, which are lazily evaluated. We can call `.compute()` to run the computation on our cluster.

In [ ]:
result.compute()

Hooray, we've run our first SQL query with Dask-SQL! Let's try out some more complex queries.

## More complex SQL examples

With Dask-SQL we can run more complex SQL statements like, for example, a groupby-aggregation:

In [ ]:
c.sql('SELECT avg(tip_amount) FROM taxi GROUP BY passenger_count').compute()

NOTE: that the equivalent operatation using the Dask DataFrame API would be:

```python
df.groupby("passenger_count").tip_amount.mean().compute()
```

We can even make plots of our SQL query results for near-real-time interactive data exploration and visualization.

In [ ]:
c.sql("""
    SELECT floor(trip_distance) AS dist, avg(fare_amount) as fare
    FROM taxi 
    WHERE trip_distance < 50 AND trip_distance >= 0 
    GROUP BY floor(trip_distance)
""").compute().plot(x="dist", y="fare");

If you would like to learn more about Dask-SQL check out the [Dask-SQL docs](https://dask-sql.readthedocs.io/) or [source code](https://github.com/nils-braun/dask-sql) on GitHub.